In [2]:
install.packages("randomForest")

Installing package into 'C:/Users/lopez/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)



package 'randomForest' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\lopez\AppData\Local\Temp\RtmpC81pUc\downloaded_packages


In [5]:
library(randomForest)
library(dplyr)
library(readr)

In [13]:
df <- read_csv('https://raw.githubusercontent.com/gabichulas/ia-uncuyo-2024/refs/heads/main/tp7-ml/data/arbolado-mendoza-dataset-train.csv', show_col_types = FALSE)
test_df <- read_csv('https://raw.githubusercontent.com/gabichulas/ia-uncuyo-2024/refs/heads/main/tp7-ml/data/arbolado-mendoza-dataset-validation.csv', show_col_types = FALSE)

In [14]:
set_1 <- df[df$inclinacion_peligrosa == 1, ]
set_0 <- df %>% filter(inclinacion_peligrosa == 0) %>% sample_n(3500)
data <- rbind(set_1, set_0)

In [15]:
x <- data[, !(colnames(data) %in% c("inclinacion_peligrosa", "id", "nombre_seccion", "area_seccion","seccion", "ultima_modificacion","circ_tronco_cm"))]
y <- factor(data$inclinacion_peligrosa)

In [63]:
trainer <- randomForest(x = x, y = y, ntree = 15000, mtry = 2)

In [64]:
predictions <- predict(trainer, newdata = test_df)

In [65]:
print(trainer)


Call:
 randomForest(x = x, y = y, ntree = 15000, mtry = 2) 
               Type of random forest: classification
                     Number of trees: 15000
No. of variables tried at each split: 2

        OOB estimate of  error rate: 29.39%
Confusion matrix:
     0    1 class.error
0 2479 1021   0.2917143
1  861 2042   0.2965897


In [71]:
library(randomForest)

# Definir una cuadrícula de hiperparámetros
tuneGrid <- expand.grid(
  ntree = c(500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 5000, 10000, 15000),
  mtry = c(2, 3, 4)
)

# Función para entrenar el modelo con diferentes combinaciones de hiperparámetros
tuneRF <- function(ntree, mtry) {
  model <- randomForest(x = x, y = y, ntree = ntree, mtry = mtry)
  return(model)
}

# Evaluar el rendimiento de cada combinación de hiperparámetros
results <- apply(tuneGrid, 1, function(params) {
  model <- tuneRF(params["ntree"], params["mtry"])
  oob_error <- model$err.rate[nrow(model$err.rate), "OOB"]
  return(oob_error)
})

# Seleccionar la mejor combinación de hiperparámetros
best_params <- tuneGrid[which.min(results), ]
best_ntree <- as.numeric(best_params["ntree"])
best_mtry <- as.numeric(best_params["mtry"])

# Entrenar el mejor modelo
best_model <- randomForest(x = x, y = y, ntree = best_ntree, mtry = best_mtry)

print(best_model)


Call:
 randomForest(x = x, y = y, ntree = best_ntree, mtry = best_mtry) 
               Type of random forest: classification
                     Number of trees: 1500
No. of variables tried at each split: 2

        OOB estimate of  error rate: 29.21%
Confusion matrix:
     0    1 class.error
0 2487 1013   0.2894286
1  857 2046   0.2952118


In [ ]:
# Generar predicciones usando el mejor modelo
predicciones <- predict(best_model, newdata = test_df)

# Crear un data frame con las predicciones
submission <- data.frame(ID = 1:length(predicciones), inclinacion_peligrosa = predicciones)

# Guardar el data frame en un archivo CSV
write.csv(submission, file = "submission.csv", row.names = FALSE)